# **(E)xtract, (T)ransform, and (L)oad of Census Data**
**Last edited:** *July 15, 2022*

**Data Sources:** [Summary Dataset](https://api.census.gov/data/2018/abscs.html), [States Dataset](https://api.census.gov/data/2018/abscs.html), [Owner Dataset](https://api.census.gov/data/2018/abscbo.html), and [Tech Dataset](https://api.census.gov/data/2018/abstcb.html)

**NOTE:** Please contact Jack Lynn (jackrlynn@gmail.com) for the `config.py` file; **this ETL notebook will not run without it!**

### **(E)tract**
1. Collect URLs (with preformatted API key) that connect to Census API from `config.py`
2. Iterate through each API call and save data to `census_data` list
   1. `requests` library is used to make the connection to API and get text
   2. Returned data is then converted to JSON format
   3. `pandas` library is then used to convert the JSON format to `pandas` format
   4. Each dataset is saved as a `DataFrame` in `census_data`

In [205]:
# Imports
import pandas as pd
import requests

# Get urls
from config import api_key
summary_url = f'https://api.census.gov/data/2018/abscb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,YIBSZFI_LABEL&for=us:*&QDESC_LABEL=SPOUSES&key={api_key}'
business_owners_url = f'https://api.census.gov/data/2018/abscbo?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_F,OWNPDEMP_PCT,OWNPDEMP_PCT_F,OWNPDEMP_S,OWNPDEMP_S_F,OWNPDEMP_PCT_S,OWNPDEMP_PCT_S_F&for=us:*&QDESC_LABEL=YRACQBUS&key={api_key}'
tech_characteristics_url = f'https://api.census.gov/data/2018/abstcb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,NSFSZFI,NSFSZFI_LABEL,FACTORS_P,FACTORS_P_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_PCT,RCPPDEMP,RCPPDEMP_PCT,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,PAYANN_PCT_S_F,TECHUSE_LABEL,TECHSELL_LABEL,MOTPRODTECH_LABEL&for=us:*&key={api_key}'
states_url = f'https://api.census.gov/data/2018/abscb?get=GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,SEX,SEX_LABEL,ETH_GROUP,ETH_GROUP_LABEL,RACE_GROUP,RACE_GROUP_LABEL,VET_GROUP,VET_GROUP_LABEL,QDESC,QDESC_LABEL,BUSCHAR,BUSCHAR_LABEL,YEAR,FIRMPDEMP,FIRMPDEMP_F,FIRMPDEMP_PCT,FIRMPDEMP_PCT_F,RCPPDEMP,RCPPDEMP_F,RCPPDEMP_PCT,RCPPDEMP_PCT_F,EMP,EMP_F,EMP_PCT,EMP_PCT_F,PAYANN,PAYANN_F,PAYANN_PCT,PAYANN_PCT_F,FIRMPDEMP_S,FIRMPDEMP_S_F,FIRMPDEMP_PCT_S,FIRMPDEMP_PCT_S_F,RCPPDEMP_S,RCPPDEMP_S_F,RCPPDEMP_PCT_S,RCPPDEMP_PCT_S_F,EMP_S,EMP_S_F,EMP_PCT_S,EMP_PCT_S_F,PAYANN_S,PAYANN_S_F,PAYANN_PCT_S,YIBSZFI_LABEL&for=state:*&QDESC_LABEL=SPOUSES&key={api_key}'

# Check connections
census_data = []
for url in [summary_url, states_url, business_owners_url, tech_characteristics_url]:
    r = requests.get(url)

    # Convert connection to JSON format
    data = r.json()

    # Save dataset as pandas
    census_data.append(pd.DataFrame(data[1:], columns=data[0]))

### **(T)ransform**
With each dataset:
1. Load in the `DataFrame` instance of that dataset, and assign a descriptive name: `summary_data`, `states_data`, `owner_data`, and `tech_data`
2. Form a name conversion dictionary to convert raw data headers to more descriptive headers
3. Make two lists of all columns that are repetative or useless, and save them as list variables
4. Make a list of columns that are numerical
5. Pass the dataset into `commonClean`, a common function that cleans the data; *see `commonClean` documentation for more info*

[Helpful document!](https://www2.census.gov/programs-surveys/abs/technical-documentation/api/ABS_API_CB-10-4-2021.pdf)

**commonClean()**: clean the dataset, including (1) dropping all redundant and useless columns, (2) removing all flag and standard error columns, (3) renaming labels to be more descriptive, (4) handling numerical nulls, and (5) casting numerical data
- ***data:*** (`DataFrame`) the dataset to be cleaned, in `pandas` format
- ***code_groups:*** (`list(str)`, def: `[]`) list of strings describing the headers that are not useful or redundant; will be removed in cleaning data
- ***rename_dict:*** (`dict(str: str)`, def: `dict()`) dictionary that includes header renames; used to make data headers more descriptive 
- ***number_cols:*** (`list(str)`) list of headers that should be considered numerical types; helps with casting data
- ***return:*** (`DataFrame`) cleaned data

In [206]:
def commonClean(data, code_groups=[], meaningless_groups=[], rename_dict=dict(), number_cols=[]):

    # Remove codes groups
    data = data.drop(columns=code_groups)

    # Remove flags and error groups
    for column in data.columns:
        if '_F' in column:
            data = data.drop(columns=[column])
        elif '_S' in column:
            data = data.drop(columns=[column])

    # Remove columns with no meaning
    data = data.drop(columns=meaningless_groups) 

    # Rename labels
    data = data.rename(columns=rename_dict)

    # Handle nulls and cast all number columns
    for column in number_cols:
        if 'Perc' in column:
            data[column].loc[data[column].notna()] = data[column].loc[data[column].notna()].astype(float)
        else:
            data[column].loc[data[column].notna()] = data[column].loc[data[column].notna()].astype(int)
        
        # Cast zeros as nulls
        data[column].loc[data[column] == 0] = None

    # Return clean data
    return data

##### ***Clean Summary Dataset***

In [207]:
# Get business dataset
summary_data = census_data[0]

# Form column header conversion
summary_label_dict = {
    'SEX_LABEL': 'Sex',
    'RACE_GROUP_LABEL': 'Race',
    'ETH_GROUP_LABEL': 'Ethnicity',
    'VET_GROUP_LABEL': 'VetStatus',
    'NAICS2017_LABEL': 'Industry',
    'YEAR': 'Year',
    'EMP': 'EmployeeCt',
    'EMP_PCT': 'EmployeeCtPerc',
    'BUSCHAR_LABEL': 'SpousalOwnershipSharing',
    'FIRMPDEMP': 'EmployerFirmCt',
    'FIRMPDEMP_PCT': 'EmployerFirmCtPerc',
    'RCPPDEMP': 'RevenueInThousands',
    'RCPPDEMP_PCT': 'RevenueInThousandsPerc',
    'PAYANN': 'AnnualPayrollInThousands',
    'PAYANN_PCT': 'AnnualPayrollInThousandsPerc',
    'YIBSZFI_LABEL': 'YearsInBusiness'
    }

# Clean data
summary_data = commonClean(summary_data,
    code_groups=['GEO_ID', 'SEX', 'RACE_GROUP', 'ETH_GROUP', 'VET_GROUP', 'NAICS2017', 'BUSCHAR'],
    meaningless_groups=['QDESC', 'QDESC_LABEL', 'us', 'NAME'],
    rename_dict=summary_label_dict,
    number_cols=['EmployerFirmCt', 'RevenueInThousands', 'EmployeeCt', 'AnnualPayrollInThousands', 
    'EmployerFirmCtPerc','RevenueInThousandsPerc', 'EmployeeCtPerc', 'AnnualPayrollInThousandsPerc'])

##### ***Firm Owner Dataset***

In [209]:
# Get firm dataset
states_data = census_data[1]

# Form column header conversion
states_label_dict = {
    'SEX_LABEL': 'Sex',
    'RACE_GROUP_LABEL': 'Race',
    'ETH_GROUP_LABEL': 'Ethnicity',
    'VET_GROUP_LABEL': 'VetStatus',
    'NAICS2017_LABEL': 'Industry',
    'YEAR': 'Year',
    'EMP': 'EmployeeCt',
    'EMP_PCT': 'EmployeeCtPerc',
    'BUSCHAR_LABEL': 'SpousalOwnershipSharing',
    'FIRMPDEMP': 'EmployerFirmCt',
    'FIRMPDEMP_PCT': 'EmployerFirmCtPerc',
    'RCPPDEMP': 'RevenueInThousands',
    'RCPPDEMP_PCT': 'RevenueInThousandsPerc',
    'PAYANN': 'AnnualPayrollInThousands',
    'PAYANN_PCT': 'AnnualPayrollInThousandsPerc',
    'NAME': 'State'
    }

# Clean data
states_data = commonClean(states_data,
    code_groups=['GEO_ID', 'SEX', 'RACE_GROUP', 'ETH_GROUP', 'VET_GROUP', 'NAICS2017', 'BUSCHAR'],
    meaningless_groups=['QDESC', 'QDESC_LABEL'],
    rename_dict=states_label_dict,
    number_cols=['EmployerFirmCt', 'RevenueInThousands', 'EmployeeCt', 'AnnualPayrollInThousands', 
    'EmployerFirmCtPerc','RevenueInThousandsPerc', 'EmployeeCtPerc', 'AnnualPayrollInThousandsPerc'])

##### ***Clean Owner Dataset***

In [210]:
# Get owner dataset
owner_data = census_data[2]

# Form column header conversion
owner_label_dict = {
    'OWNER_SEX_LABEL': 'OwnerSex',
    'OWNER_RACE_LABEL': 'OwnerRace',
    'OWNER_ETH_LABEL': 'OwnerEthnicity',
    'OWNER_VET_LABEL': 'OwnerVetStatus',
    'NAICS2017_LABEL': 'Industry',
    'YEAR': 'Year',
    'OWNCHAR_LABEL': 'OwnedSince',
    'OWNPDEMP': 'OwnerCt',
    'OWNPDEMP_PCT': 'OwnerCtPerc'
    }

# Clean data
owner_data = commonClean(owner_data,
    code_groups=['GEO_ID', 'OWNER_SEX', 'OWNER_RACE', 'OWNER_ETH', 'OWNER_VET', 'NAICS2017', 'OWNCHAR'],
    meaningless_groups=['QDESC', 'QDESC_LABEL', 'us', 'NAME'],
    rename_dict=owner_label_dict,
    number_cols=['OwnerCt', 'OwnerCtPerc'])

##### ***Clean Tech Dataset***

In [211]:
# Get tech dataset
tech_data = census_data[3]

# Form column header conversion
tech_label_dict = {
    'SEX_LABEL': 'Sex',
    'RACE_GROUP_LABEL': 'Race',
    'ETH_GROUP_LABEL': 'Ethnicity',
    'VET_GROUP_LABEL': 'VetStatus',
    'NAICS2017_LABEL': 'Industry',
    'YEAR': 'Year',
    'NSFSZFI_LABEL': 'CompanySize',
    'FACTORS_P_LABEL': 'TechAdverseReason',
    'RCPPDEMP': 'RevenueInThousands',
    'RCPPDEMP_PCT': 'RevenueInThousandsPerc',
    'PAYANN': 'AnnualPayrollInThousands',
    'PAYANN_PCT': 'AnnualPayrollInThousandsPerc',
    'EMP': 'EmployeeCt',
    'EMP_PCT': 'EmployeeCtPerc',
    'FIRMPDEMP': 'EmployerFirmCt',
    'FIRMPDEMP_PCT': 'EmployerFirmCtPerc',
    'TECHUSE_LABEL': 'TechUseExtent',
    'TECHSELL_LABEL': 'TechProduction',
    'MOTPRODTECH_LABEL': 'TechMotivation'
    }

# Clean data
tech_data = commonClean(tech_data,
    code_groups=['GEO_ID', 'SEX', 'RACE_GROUP', 'ETH_GROUP', 'VET_GROUP', 'NAICS2017', 'NSFSZFI', 'FACTORS_P'],
    meaningless_groups=['us', 'NAME'],
    rename_dict=tech_label_dict,
    number_cols=['RevenueInThousands', 'RevenueInThousandsPerc', 'AnnualPayrollInThousands',
        'AnnualPayrollInThousandsPerc', 'EmployeeCt', 'EmployeeCtPerc', 'EmployerFirmCt', 'EmployerFirmCtPerc'])

### **(T)ransform**
Pass each `DataFrame` into `to_csv` class function, saving to `data` folder

In [212]:
# Save all data to CSVs
summary_data.to_csv('data/summary_dataset.csv')
states_data.to_csv('data/states_dataset.csv')
owner_data.to_csv('data/owner_dataset.csv')
tech_data.to_csv('data/tech_dataset.csv')